In [1]:
import pymongo
import dns
import pandas as pd
import io
import datetime

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017")
db = client.traffic

In [10]:
def getGantryInput(gantryID, timeStamp, timeRange):
    begin=datetime.datetime.strptime(timeStamp, "%Y/%m/%d %H:%M") #"%Y-%m-%d %H:%M") # Time Format2019-06-24 06:40
    end=begin+ datetime.timedelta(minutes = timeRange) #timeRange in minutes
    coll = db['TDCS-M03A']
    cursor = coll.find({"GantryId":gantryID, "TimeInterval": {"$gte": begin, "$lte": end}},{"TimeInterval":1, "GantryId":1 ,"VehicleType": 1,"交通量":1})
    df =  pd.DataFrame(list(cursor))
    del df['_id']
    return df

In [42]:
df=getGantryInput('01F2089N','2019/06/25 08:23',30)

In [43]:
df

,TimeInterval,GantryId,VehicleType,交通量
0,2019-06-25 08:45:00,01F2089N,31,135
1,2019-06-25 08:45:00,01F2089N,32,71
2,2019-06-25 08:45:00,01F2089N,41,4
3,2019-06-25 08:45:00,01F2089N,42,29
4,2019-06-25 08:45:00,01F2089N,5,11
5,2019-06-25 08:30:00,01F2089N,31,148
6,2019-06-25 08:30:00,01F2089N,32,72
7,2019-06-25 08:30:00,01F2089N,41,7
8,2019-06-25 08:30:00,01F2089N,42,14
9,2019-06-25 08:30:00,01F2089N,5,12


In [44]:
def inputGantry(gantryID, timeStamp, timeRange):
    begin=datetime.datetime.strptime(timeStamp, "%Y/%m/%d %H:%M") #"%Y-%m-%d %H:%M") # Time Format2019-06-24 06:40
    end=begin+ datetime.timedelta(minutes = timeRange) #timeRange in minutes
    coll = db['TDCS-M03A']
    match={"GantryId":gantryID, "TimeInterval": {"$gte": begin, "$lte": end}}
    groupby = 'VehicleType'
    group = {'_id': "$%s" % (groupby if groupby else None),
             'total': {'$sum': '$交通量'}}
    result = coll.aggregate([
            { "$match": match },
            { "$group": group}])
    
    df =  pd.DataFrame(list(result))
    return df

In [46]:
df=inputGantry('01F2089N','2019/06/25 08:23',30)
df

,_id,total
0,5,103
1,32,414
2,41,32
3,31,783
4,42,139


In [55]:
def getGantryOff(gantryID, timeStamp, timeRange):
    begin=datetime.datetime.strptime(timeStamp, "%Y/%m/%d %H:%M") #"%Y-%m-%d %H:%M") # Time Format2019-06-24 06:40
    end=begin+ datetime.timedelta(minutes = timeRange) #timeRange in minutes
    coll = db['TDCS-M08A']
    cursor = coll.find({"GantryTo":gantryID, "TimeInterval": {"$gte": begin, "$lte": end}},{"TimeInterval":1, "GantryTo":1 ,"VehicleType": 1,"交通量":1})
    df =  pd.DataFrame(list(cursor))
    #del df['_id']
    return df

In [56]:
df=getGantryOff('01F2089N','2019/06/25 08:23',30)
df

,_id,TimeInterval,GantryTo,VehicleType,交通量
0,5f21522a3399a14e46f80382,2019-06-25 08:45:00,01F2089N,31,12
1,5f21522a3399a14e46f80383,2019-06-25 08:45:00,01F2089N,32,10
2,5f21522a3399a14e46f80384,2019-06-25 08:45:00,01F2089N,42,3
3,5f21522a3399a14e46f803b1,2019-06-25 08:45:00,01F2089N,31,4
4,5f21522a3399a14e46f803b2,2019-06-25 08:45:00,01F2089N,32,7
...,...,...,...,...,...
231,5f21522a3399a14e46f89347,2019-06-25 08:35:00,01F2089N,31,2
232,5f21522a3399a14e46f89348,2019-06-25 08:35:00,01F2089N,32,1
233,5f21522a3399a14e46f89399,2019-06-25 08:35:00,01F2089N,31,1
234,5f21522a3399a14e46f8939a,2019-06-25 08:35:00,01F2089N,32,1


In [60]:
def offInterchange(gantryID, timeStamp, timeRange):
    begin=datetime.datetime.strptime(timeStamp, "%Y/%m/%d %H:%M") #"%Y-%m-%d %H:%M") # Time Format2019-06-24 06:40
    end=begin+ datetime.timedelta(minutes = timeRange) #timeRange in minutes
    coll = db['TDCS-M08A']
    match={"GantryTo":gantryID, "TimeInterval": {"$gte": begin, "$lte": end}}
    groupby = 'VehicleType'
    group = {'_id': "$%s" % (groupby if groupby else None),
             'total': {'$sum': '$交通量'}}
    result = coll.aggregate([
            { "$match": match },
            { "$group": group}])
    
    df =  pd.DataFrame(list(result))
    return df

In [61]:
df=offInterchange('01F2089N','2019/06/25 08:23',30)
df

,_id,total
0,42,95
1,5,79
2,32,157
3,41,5
4,31,251


In [63]:
df=offInterchange('01F2089N','2019/06/25 07:53',30)
df

,_id,total
0,42,72
1,5,53
2,32,142
3,41,14
4,31,232


In [58]:
def goInterchange(gantryFrom, gantryTo , timeStamp, timeRange):
    fromGntry=inputGantry(gantryFrom, timeStamp, timeRange)
    toGntry=inputGantry(gantryTo , timeStamp, timeRange)
    offIntr=offInterchange(gantryFrom,timeStamp,timeRange)
    
    return toGntry-fromGntry+offIntr

,_id,total
0,5,103
1,32,414
2,41,32
3,31,783
4,42,139


In [ ]:
df1.set_index('t1').add(df2.set_index('t2'), fill_value=0)
          .reset_index()
          .rename(columns={'index':'t'})